In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv1D, Dropout, Dense, Activation, BatchNormalization, Input, Flatten, MaxPooling1D
from tensorflow.keras.losses import CategoricalCrossentropy
import pandas as pd
from pandas_datareader import DataReader
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
import seaborn as sns
scaler = StandardScaler()

In [2]:
symbols = ['AAPL', 'TSLA', 'NVDA', 'MSFT', 'GOOGL', 'F']
start = '10-10-2018'
end = '10-10-2020'
api = 'yahoo'
stocks = {}
for symbol in symbols:
    stocks[symbol] = DataReader(symbol, api, start, end)

In [3]:
for symbol in stocks:
    data = stocks[symbol]
    data['Buy'] = data['Close'] > data['Open']
    data['Buy'] = data['Buy'].astype(int)
    data['Sell'] = data['Close'] < data['Open']
    data['Sell'] = data['Sell'].astype(int)

In [4]:
ldf = 8 #largest dilation factor
feature_count = 100 #get 30 unique features
final_output = 1 #only one output feature, a number in the range of [0-1] with 1 meaning hold/buy and 0 meaning (don't buy)/sell
feature_window = 10 #how big the window is for each feature
train_split = feature_window+(feature_window-1)*(ldf-1) #calculate the receptive field of the TCN
features = [feature_count]*(int(math.log(ldf)/math.log(2)) + 1)

In [5]:
train_y = []
train_x = []
valid_x = []
valid_y = []
for symbol in stocks:
    data = stocks[symbol]
    outdata = data[['Buy', 'Sell']].values
    
    indata = data[['Volume']].copy()
    indata['Day Avg'] = data[['Open', 'High', 'Low']].mean(axis=1)
    indata = scaler.fit_transform(indata.values)
    opendata = indata[:,1]
    
    train_y.append(outdata[:train_split].reshape((1, train_split, 2)))
    valid_y.append(outdata[train_split:2*train_split].reshape((1, train_split, 2)))
    
    train_x.append(opendata[:train_split].reshape((1, train_split, 1)))
    valid_x.append(opendata[train_split:2*train_split].reshape((1, train_split, 1)))
train_x = np.array(train_x)
train_y = np.array(train_y)
valid_x = np.array(valid_x)
valid_y = np.array(valid_y)

In [6]:
print(train_y.shape)
print(valid_y.shape)
print(train_x.shape)
print(valid_x.shape)

(6, 1, 73, 2)
(6, 1, 73, 2)
(6, 1, 73, 1)
(6, 1, 73, 1)


In [25]:
class TCN(Model):
    def __init__(self, features, kernel_size, dropout=0.2, activation='relu'):
        super(TCN, self).__init__()
        self.convs1 = []
        self.convs2 = []
        self.features = features
        self.levels = len(features)
        for i in range(self.levels):
            self.convs1.append(Conv1D(features[i], kernel_size, 1, 'causal', dilation_rate=2**i))
            self.convs2.append(Conv1D(features[i], kernel_size, 1, 'causal', dilation_rate=2**i))
        self.maxp = MaxPooling1D(features[0], strides = features[0], padding='same')
        self.maxp2 = MaxPooling1D(int(features[0]/2), strides = int(features[0]/2), padding = 'same')
        self.batch_norm = BatchNormalization(1)
        self.activation = Activation(activation)
        self.finalactivation = Activation('softmax')
        self.drop = Dropout(dropout, (1, train_split, 1))
    def call(self, inputs, training = False):
        length = inputs.shape[1]
        for i in range(self.levels):
            x = self.convs1[i](inputs)
            x = tf.reshape(x, (1, length*features[i], 1))
            x = self.maxp(x)
            x = self.batch_norm(x, training)
            x = self.activation(x)
            x = self.drop(x, training)

            x = self.convs2[i](x)
            x = tf.reshape(x, (1, length*features[i], 1))
            if i == self.levels-1:
                x = self.maxp2(x)
                x = tf.reshape(x, (1, train_split, 2))
                x = self.batch_norm(x, training)
                inputs = self.finalactivation(x + inputs)
            else:
                x = self.maxp(x)
                x = self.batch_norm(x, training)
                x = self.activation(x)
                x = self.drop(x, training)
                inputs = self.activation(x + inputs)
        return inputs

In [26]:
model = TCN(features, feature_window, 0.4, 'sigmoid')
model.compile(optimizer = 'rmsprop', loss=CategoricalCrossentropy(), metrics=tf.keras.metrics.CategoricalCrossentropy())
for i in range(1):
    model.fit(x=train_x[i], y=train_y[i], epochs=50, validation_data=(valid_x[i], valid_y[i]))

Epoch 1/50
1/1 [==============================] - 1s 523ms/step - loss: 0.7853 - categorical_crossentropy: 0.7853 - val_loss: 0.6927 - val_categorical_crossentropy: 0.6927
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 0.7702 - categorical_crossentropy: 0.7702 - val_loss: 0.6933 - val_categorical_crossentropy: 0.6933
Epoch 3/50
1/1 [==============================] - 0s 21ms/step - loss: 0.7509 - categorical_crossentropy: 0.7509 - val_loss: 0.6933 - val_categorical_crossentropy: 0.6933
Epoch 4/50
1/1 [==============================] - 0s 21ms/step - loss: 0.8112 - categorical_crossentropy: 0.8112 - val_loss: 0.6940 - val_categorical_crossentropy: 0.6940
Epoch 5/50
1/1 [==============================] - 0s 20ms/step - loss: 0.7430 - categorical_crossentropy: 0.7430 - val_loss: 0.6939 - val_categorical_crossentropy: 0.6939
Epoch 6/50
1/1 [==============================] - 0s 17ms/step - loss: 0.7221 - categorical_crossentropy: 0.7221 - val_loss: 0.6940 - val_catego

Epoch 49/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6388 - categorical_crossentropy: 0.6388 - val_loss: 0.7002 - val_categorical_crossentropy: 0.7002
Epoch 50/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6853 - categorical_crossentropy: 0.6853 - val_loss: 0.7014 - val_categorical_crossentropy: 0.7014


In [24]:
out = model.predict(train_x[1])
for i in range(out.shape[1]):
    if out[0, i, 0] > .5:
        if i == 0:
            print('[[[1 0]')
        elif i == out.shape[1] - 1:
            print('  [1 0]]]')
        else:
            print('  [1 0]')
    else:
        if i == 0:
            print('[[[0 1]')
        elif i == out.shape[1] -1:
            print('  [0 1]]]')
        else:
            print('  [0 1]')
print(train_y[1])

[[[0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]]]
[[[0 1]
  [0 1]
  [0 1]
  [1 0]
  [1 0]
  [0 1]
  [0 1]
  [0 1]
  [1 0]
  [1 0]
  [0 1]
  [0 1]
  [1 0]
  [0 1]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [0 1]
  [1 0]
  [1 0]
  [1 0]
  [1 0]
  [0 1]
  [1 0]
  [0 1]
  [0 1]
  [1 0]
  [1 0]
  [1 0]
  [0 1]
  [1 0]
  [0 1]
  [1 0]
  [1 0]
  [0 1]
  [1 0]
  [0 1]
  [0 1]
  [1 0]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [0 1]
  [1 0]
  [0 1